**Chat Model**


In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI
from langchain.llms.loading import load_llm


# chat = OpenAI(
#     temperature=0.1,
#     max_tokens=450,
#     model="gpt-3.5-turbo-16k",
#     # streaming=True,
#     # callbacks=[StreamingStdOutCallbackHandler()],
# )
# chat.save("model.json")

In [ ]:
chat = load_llm("model.json")
chat